In [1]:
import argparse
import os
import numpy as np
import math
import sys
import torch
import torch.distributions as distributions
import torchvision.transforms as transforms
from torchvision.utils import save_image
import torch.utils.data as Data
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch
from scipy.io import savemat
import scipy.io as sio
import sys
import importlib
import datetime
import itertools
import pickle
import scipy
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy.random import random
from scipy.linalg import sqrtm
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

In [2]:
data_dir = "data/case30"
case_dir = os.path.join(data_dir)
number = 20000
train_index = 12000
valid_index = 16000
saveFile = "data/case30/LR_out/"

In [3]:
#input data
mat_input = os.path.join(case_dir, 'inputdata_sample.mat')
PQ_injection = np.transpose(scipy.io.loadmat(mat_input)['sampledata'])[0:number,:]
P_injection = PQ_injection.real
Q_injection = PQ_injection.imag
# print(Q_injection.shape)
pv = [1-1,2-1,13-1,22-1,23-1,27-1]
slack = 1-1
P_injection = np.delete(P_injection,slack,1)
Q_injection = np.delete(Q_injection,pv,1)
# print(Q_injection.shape)
PQ_injection = np.concatenate([P_injection, Q_injection], axis=1)       
print(PQ_injection.shape)
#output data
mat_ang = os.path.join(case_dir, 'voltage_ang_reduced.mat')
mat_mag = os.path.join(case_dir, 'voltage_mag_reduced.mat')
voltage_ang = np.transpose(scipy.io.loadmat(mat_ang)['voltage_ang_reduced'])[0:number,:]
voltage_mag = np.transpose(scipy.io.loadmat(mat_mag)['voltage_mag_reduced'])[0:number,:]
voltage =  np.concatenate([voltage_ang,voltage_mag], axis=1) 
yTrain = torch.from_numpy(voltage[0:train_index,:])
xTrain = torch.from_numpy(PQ_injection[0:train_index,:])

yValid = torch.from_numpy(voltage[train_index:valid_index,:])
xValid = torch.from_numpy(PQ_injection[train_index:valid_index,:])

yTest = torch.from_numpy(voltage[valid_index:number,:])
xTest = torch.from_numpy(PQ_injection[valid_index:number,:])

dataset = torch.utils.data.TensorDataset(xTrain, yTrain)
print(xValid.shape)
print(xTrain.shape)
print(xTest.shape)

(20000, 53)
torch.Size([4000, 53])
torch.Size([12000, 53])
torch.Size([4000, 53])


In [4]:
print(xTest)
print(yTest)

tensor([[ 0.4118, -0.0481, -0.0867,  ..., -0.0166, -0.0508, -0.1126],
        [ 0.7995, -0.0416, -0.0886,  ..., -0.0131, -0.0724, -0.0762],
        [ 0.5675, -0.0451, -0.0884,  ..., -0.0196, -0.0453, -0.0785],
        ...,
        [ 0.6787, -0.0395, -0.0917,  ..., -0.0152, -0.0627, -0.0939],
        [ 0.7119, -0.0383, -0.0912,  ..., -0.0119, -0.0818, -0.0694],
        [ 0.6999, -0.0431, -0.0938,  ..., -0.0116, -0.0865, -0.0769]],
       dtype=torch.float64)
tensor([[-0.0316, -0.0624, -0.0742,  ...,  0.9527,  0.9246,  0.8888],
        [ 0.0144, -0.0127, -0.0139,  ...,  0.9671,  0.9257,  0.9083],
        [-0.0171, -0.0495, -0.0586,  ...,  0.9580,  0.9403,  0.9161],
        ...,
        [ 0.0132, -0.0063, -0.0062,  ...,  0.9707,  0.9248,  0.8988],
        [-0.0187, -0.0587, -0.0701,  ...,  0.9495,  0.9227,  0.9099],
        [ 0.0010, -0.0272, -0.0315,  ...,  0.9637,  0.9172,  0.9028]],
       dtype=torch.float64)


In [54]:
cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
reg = Ridge(alpha = 0.0001).fit(xTrain, yTrain)
initial_weight_original = reg.coef_
initial_bias_original = reg.intercept_

In [52]:
print(initial_weight_original)
print(initial_bias_original)

[[ 5.60702363e-02  6.33514820e+08 -1.81420605e+08 ...  1.12455925e+07
   7.89758287e+07 -1.15172864e+08]
 [ 3.35620452e-02 -1.68096358e+07  7.08865947e+06 ...  5.82388469e+05
   1.82397497e+06 -3.92072345e+06]
 [ 4.08150800e-02 -5.12817713e+06  4.50923105e+06 ...  1.08625266e+06
   4.59968589e+06 -8.39305164e+06]
 ...
 [-4.41134305e-04 -1.72961416e+08  5.42843335e+07 ... -1.22995178e+06
  -1.33724144e+07  1.68671470e+07]
 [-1.17775118e-05 -1.01169872e+09  2.93154978e+08 ... -1.66294845e+07
  -1.20205921e+08  1.73397189e+08]
 [-1.01061210e-05 -1.44410416e+09  4.16355276e+08 ... -2.45484297e+07
  -1.75193416e+08  2.53935353e+08]]
[ 190186.22102219  497456.36094835  670125.83744021  482879.90135177
  854429.84011777  792010.90992221  990468.67972027 1533492.67931789
 1647234.38050711 1533606.24184042 1477203.4790188  1449799.15891404
 1567054.32397042 1637032.06312063 1662870.27277613 1652817.20614092
 1670211.01155078 1666899.61655542 1663880.10598993 1802788.19117273
 1849283.00810575 1

In [55]:
yhatTest = torch.from_numpy(reg.predict(xTest))
print(yhatTest.shape)

torch.Size([4000, 53])


In [57]:
rmse_test = torch.sqrt(torch.mean((yhatTest  - yTest.float())**2,axis = 0))
print(rmse_test.shape)
rmse_ang_test = torch.mean(rmse_test[0:29])
rmse_mag_test = torch.mean(rmse_test[29:53])
print(rmse_ang_test)
print(rmse_mag_test)

torch.Size([53])
tensor(0.0004, dtype=torch.float64)
tensor(0.0002, dtype=torch.float64)


In [7]:
yHatTest = torch.from_numpy(reg.predict(xTest))
print(yHatTest.shape)

torch.Size([4000, 53])


In [10]:
print(yHatTest)

tensor([[-0.03154915, -0.06235278, -0.07415879,  ...,  0.95265453,
          0.92456699,  0.88881422],
        [ 0.01452604, -0.01251116, -0.01367367,  ...,  0.96750754,
          0.92553970,  0.90824237],
        [-0.01705299, -0.04936830, -0.05846079,  ...,  0.95810817,
          0.94071741,  0.91640618],
        ...,
        [ 0.01353708, -0.00600337, -0.00579336,  ...,  0.97194744,
          0.92464145,  0.89853910],
        [-0.01856125, -0.05847220, -0.06981204,  ...,  0.94958592,
          0.92260769,  0.90997718],
        [ 0.00097786, -0.02718318, -0.03148452,  ...,  0.96363609,
          0.91695197,  0.90259371]], dtype=torch.float64)


In [8]:
torch.set_printoptions(precision=8)
rmse_test = torch.sqrt(torch.mean((yHatTest  - yTest.float())**2,axis = 0))
print(rmse_test.shape)
rmse_ang_test = torch.mean(rmse_test[0:29])
rmse_mag_test = torch.mean(rmse_test[29:53])
print(rmse_ang_test)
print(rmse_mag_test)

torch.Size([53])
tensor(0.00036650, dtype=torch.float64)
tensor(0.00017872, dtype=torch.float64)


In [9]:
np.save(saveFile+'rmse_test_best',rmse_test)
np.save(saveFile+'yHatTest_best',yHatTest.detach().numpy())
np.save(saveFile+'yHatTest',yHatTest)